In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, sys
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import spearmanr
import matplotlib.dates as mdates
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timedelta
import seaborn as sns
import copy

sys.path.append('../utils/')

from utils import *
from analysis_utils import *

In [ ]:
"""
Necessary data (both weekly and monthly):
- Shocks
- Confounds
- Trends outcomes
- KSU
"""

prefix = '../data/prepared/merged/'
units = 'weeks'

if units == 'weeks':
    bonf_denom = 7
elif units == 'months':
    bonf_denom = 6
else:
    assert 0

do_controls = False
do_primary = True

# Run primary analyses
pval_thresh_1 = 0.05/bonf_denom

month_tests = 6*15*3 #intvns * outcomes * [assoc, contemp, lagged]
week_tests = 7*12*3 # intvns * outcomes * [assoc, contemp lagged]

all_tests = month_tests + week_tests

pval_thresh_2 = 0.05/all_tests#(month_tests + week_tests)

# Settings for non-bin models
do_differencing = True
include_time = False

# Settings for both
add_L = True
normalize = True
include_month = True

# Settings for bin models
do_differencing_bin = False
include_time_bin = True
duration_months = 5
bin_slope = True

models_to_run = ['assoc', 'contemp', 'lagged']

# KSU params
ksu_lag = None

verbose=True


netflix_release_dates = {'tgc': [2019, 10, 16], 'fok': [2011, 5, 6], 'okja': [2017, 6, 28],
                'wth': [2017, 6, 16], 'cowspiracy': [2015, 9, 15], 'owth': [2017, 6, 16], 
                         'yawye': [2024, 1, 1]}

restrictions = {'StewartMilk': ['cowspiracy', 'wth', 'okja', 'owth', 'all_docs'], 
                'StewartPBMilk': ['cowspiracy', 'wth', 'okja', 'owth', 'all_docs'],
                'Zhao': ['wth', 'okja', 'owth', 'tgc', 'all_docs']}

binary_analysis_dct = {'Zhao': ['owth', 'wth', 'okja', 'tgc'], 'NeuhoferLusk': ['tgc'],
                      'StewartMilk': ['cowspiracy', 'owth', 'wth', 'okja'], 
                       'StewartPBMilk': ['cowspiracy', 'owth', 'wth', 'okja']}

normalized = pd.read_csv(prefix + 'merged_' + units + '.csv')

if do_controls:
    normalized_controls = pd.read_csv(prefix + 'merged_controls_' + units + '.csv')

assert units in ['weeks', 'months', 'days']
assert len(set(units) & set(prefix)) !=0 

#Lists
common = ['ds', 'Time', 'Month']

all_docs = ['tgc', 'wth', 'fok', 'cowspiracy', 'okja', 'yawye']
all_ts_albums = ['reputation', 'ts_1989', 'lover', 'speak_now', 'red']
all_climate = ['climate', 'climate_change', 'sustainability']
all_ts_outcomes = ['taylor_swift', 'taylor_swift_lyrics', 'taylor_swift_songs',
                  'taylor_swift_tour', 'taylor_swift_album']

all_drake_albums = ['scorpion', 'take_care', 'views', 'nwts', 'tml']
all_drake_outcomes = ['drake', 'drake_lyrics', 'drake_songs', 'drake_tour', 'drake_album']


if units == 'months':
    all_primary_outcomes = ['plant_based_plus_plant_based', 'vegan', 'vegetarian', 
                           'ksu_beef', 'ksu_pork', 'ksu_chicken']
    
    all_primary_outcomes += ['StewartMilk',  
                            'StewartPBMilk', 'Zhao'] 

    all_secondary_outcomes = ['vegan_informative', 'vegetarian_informative',
           'plant_based_informative', 'vegan_behavior', 'vegetarian_behavior',
           'plant_based_behavior']
    
    all_outcomes = all_primary_outcomes + all_secondary_outcomes 
    
    intvns = ['fok', 'cowspiracy', 'owth', 'tgc', 'yawye', 'all_docs']
    
elif units == 'weeks':
    all_primary_outcomes = ['plant_based_plus_plant_based','vegan', 'vegetarian',
                            'StewartMilk', 
                            'StewartPBMilk',
                            'Zhao']   

    all_secondary_outcomes = ['vegan_informative', 'vegetarian_informative',
           'plant_based_informative', 'vegan_behavior', 'vegetarian_behavior',
           'plant_based_behavior']
    
    all_outcomes = all_primary_outcomes + all_secondary_outcomes
    intvns = ['fok', 'cowspiracy', 'okja', 'wth', 'tgc', 'yawye', 'all_docs'] #'okja', 'wth'        
    
else:
    assert 0
        
# Outcomes, interventions, and models to run
test_outcomes = all_outcomes
test_intvns =  intvns

models = ['assoc', 'contemp', 'lagged', 'bin']
for model in models_to_run:
    assert model in models

#run_name = 'bin_{num}months_slope{val}'.format(num=duration_months, val=bin_slope)

run_name = 'test'#'ksu_lag_' + str(ksu_lag) + 'assoc_contemp_lagged_2024'

print(run_name)


In [ ]:
# Shift KSU
if ksu_lag:
    normalized['ksu_chicken'] = normalized['ksu_chicken'].shift(-1*ksu_lag)
    normalized['ksu_pork'] = normalized['ksu_pork'].shift(-1*ksu_lag)
    normalized['ksu_beef'] = normalized['ksu_beef'].shift(-1*ksu_lag)

In [ ]:
# Merge for controls
# 522
if do_controls:
    merged_for_controls = normalized.merge(normalized_controls, on=common)
    run_controls(merged_for_controls.copy(), pval_thresh=0.05, PS=True, PS_logistic=False, 
                 fit_method='GLSAR', model='lagged', bonf_denom=522,
                    add_L=False, difference=True, normalize=True, include_month=True,
                include_time=False, verbose=False)
    
    """
    run_controls_binary(merged_for_controls.copy(), PS=False, PS_logistic=False, fit_method='GLSAR', model='bin', bonf_denom=5,
                 add_L=True,
                duration_months=12,
                difference=False, verbose=False)
    """

In [ ]:
# Dfs for heatmaps
heat_dfs = {}
annot_dfs = {}

intvn_map = {'fok': 'FOK', 'cowspiracy': 'Cowspiracy', 'wth': 'WTH', 'okja': "Okja", 'owth': 
             'Okja/WTH', 'tgc': 'TGC', 'yawye': 'YAWYE', 'all_docs': 'All'}
outcome_map = {'vegan': "Searches: `Vegan'", 'vegetarian': "Searches: `Vegetarian'", 
               'plant_based_plus_plant_based': "Searches: `Plant based'",
              'ksu_beef': 'Beef Demand',
              'ksu_chicken': 'Chicken Demand',
              'ksu_pork': 'Pork Demand',
              'Zhao': 'Zhao',
              'NeuhoferLusk': 'NeuhoferLusk',
              'StewartPBMilk': 'StewartPBMilk',
              'StewartMilk': 'StewartMilk',
               'vegan_informative': "Searches: `Vegan', Informative",
           'vegetarian_informative': "Searches: `Vegetarian', Informative", 
               'plant_based_informative': "Searches: `Plant based', Informative", 
               'vegan_behavior': "Searches: `Vegan', Behavior",
           'vegetarian_behavior': "Searches: `Vegetarian', Behavior", 
               'plant_based_behavior': "Searches: `Plant based', Behavior"}

coef_dcts = {}
annot_dcts = {}
pval_dcts = {}
se_dcts = {}

for model in models:
    coef_dcts[model] = {}
    annot_dcts[model] = {}
    pval_dcts[model] = {}
    se_dcts[model] = {}



for model in models:
    for intvn in intvns:
        coef_dcts[model][intvn_map[intvn]] = {}
        annot_dcts[model][intvn_map[intvn]] = {}
        pval_dcts[model][intvn_map[intvn]] = {}
        se_dcts[model][intvn_map[intvn]] = {}
        for outcome in all_outcomes:
            coef_dcts[model][intvn_map[intvn]][outcome_map[outcome]] = np.nan
            annot_dcts[model][intvn_map[intvn]][outcome_map[outcome]] = ''
            pval_dcts[model][intvn_map[intvn]][outcome_map[outcome]] = np.nan
            se_dcts[model][intvn_map[intvn]][outcome_map[outcome]] = np.nan
    heat_dfs[model] = pd.DataFrame(coef_dcts[model])
    annot_dfs[model] = pd.DataFrame(annot_dcts[model])

In [ ]:
def eval_pval(pval, thresh1, thresh2, sens_pvals=None):
    if pval < thresh2:
        if sens_pvals:
            for sens_pval in sens_pvals:
                if sens_pval >= thresh2:
                    return '**'
            return '***'
        else:
            return '**'
    elif pval < thresh1:
        return '*'
    else:
        return ''

In [ ]:
if do_primary:
    #for outcome in ['ksu_beef', 'ksu_pork', 'ksu_chicken']:
    sens_add_L = not add_L
    for outcome in test_outcomes:
        for intvn in test_intvns:
            if outcome in restrictions and intvn not in restrictions[outcome]:
                continue
            
            print('intvn: ', intvn)
            
            if 'assoc' in models_to_run:
                print('Association: ')
                assoc_pval, assoc_beta, assoc_se = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False, 
                                     fit_method='GLSAR', model='assoc', 
                                     add_L=False, difference=do_differencing, include_time=include_time, 
                                     include_month=False, normalize=normalize, verbose=verbose, units=units)

                print('beta, pval: ', assoc_beta, assoc_pval)
                coef_dcts['assoc'][intvn_map[intvn]][outcome_map[outcome]] = assoc_beta
                annot_dcts['assoc'][intvn_map[intvn]][outcome_map[outcome]] = eval_pval(assoc_pval, pval_thresh_1, pval_thresh_2)
                se_dcts['assoc'][intvn_map[intvn]][outcome_map[outcome]] = assoc_se
                pval_dcts['assoc'][intvn_map[intvn]][outcome_map[outcome]] = assoc_pval
            
            if 'contemp' in models_to_run:
                print('Contemporaneous: ')
                contemp_pval, contemp_beta, contemp_se = run_primary_analyses(normalized.copy(), intvn, outcome, PS=True, 
                                    fit_method='GLSAR', model='contemp', 
                                     add_L=add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)

                contemp_pval_sens1, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=True, 
                                    fit_method='GLSAR', model='contemp', 
                                     add_L=sens_add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)            

                contemp_pval_sens2, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False, 
                                    fit_method='GLSAR', model='contemp', 
                                     add_L=add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)   

                print('beta, pval: ', contemp_beta, contemp_pval)
                coef_dcts['contemp'][intvn_map[intvn]][outcome_map[outcome]] = contemp_beta
                se_dcts['contemp'][intvn_map[intvn]][outcome_map[outcome]] = contemp_se
                contemp_pvals_sens = [contemp_pval_sens1, contemp_pval_sens2]
                print('contemp_pvals_sens: ', contemp_pvals_sens)
                annot_dcts['contemp'][intvn_map[intvn]][outcome_map[outcome]] = eval_pval(contemp_pval, pval_thresh_1, pval_thresh_2, contemp_pvals_sens)
                pval_dcts['contemp'][intvn_map[intvn]][outcome_map[outcome]] = contemp_pval
            
            if 'lagged' in models_to_run:
                print('Lagged: ')
                lagged_pval, lagged_beta, lagged_se = run_primary_analyses(normalized.copy(), intvn, outcome, PS=True, 
                                     fit_method='GLSAR', model='lagged', 
                                     add_L=add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)

                lagged_pval_sens1, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=True, 
                                     fit_method='GLSAR', model='lagged', 
                                     add_L=sens_add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)

                lagged_pval_sens2, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False, 
                                     fit_method='GLSAR', model='lagged', 
                                     add_L=add_L, difference=do_differencing, include_time=include_time, 
                                     include_month=include_month, normalize=normalize, verbose=verbose, units=units)

                print('beta, pval: ', lagged_beta, lagged_pval)
                coef_dcts['lagged'][intvn_map[intvn]][outcome_map[outcome]] = lagged_beta
                se_dcts['lagged'][intvn_map[intvn]][outcome_map[outcome]] = lagged_se
                lagged_pvals_sens = [lagged_pval_sens1, lagged_pval_sens2]
                print('lagged_pvals_sens: ', lagged_pvals_sens)
                annot_dcts['lagged'][intvn_map[intvn]][outcome_map[outcome]] = eval_pval(lagged_pval, pval_thresh_1, pval_thresh_2, lagged_pvals_sens)
                pval_dcts['lagged'][intvn_map[intvn]][outcome_map[outcome]] = lagged_pval

            print('Bin: ')
            if 'bin' in models_to_run:
                if (outcome in binary_analysis_dct) and (intvn not in binary_analysis_dct[outcome]):
                    continue

                if intvn == 'all_docs':
                    continue
                    
                bin_pval, bin_beta, bin_se = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False, 
                                                          PS_logistic=False,
                                     fit_method='GLSAR', model='bin', 
                                     add_L=add_L, difference=do_differencing_bin, include_time=include_time_bin, 
                                     include_month=include_month, duration_months=duration_months, 
                                                          normalize=normalize, verbose=verbose, units=units)

                bin_pval_sens1, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False,
                                                   PS_logistic=False,
                                     fit_method='GLSAR', model='bin', 
                                     add_L=sens_add_L, difference=do_differencing_bin, include_time=include_time_bin, 
                                     include_month=include_month, duration_months=duration_months, 
                                                            normalize=normalize, verbose=verbose, units=units)

                bin_pval_sens2, _, _ = run_primary_analyses(normalized.copy(), intvn, outcome, PS=False, 
                                    PS_logistic=False,
                                     fit_method='GLSAR', model='bin', 
                                     add_L=add_L, difference=do_differencing_bin, include_time=include_time_bin, 
                                     include_month=include_month, duration_months=duration_months, 
                                                            normalize=normalize, verbose=verbose, units=units)

                print('beta, pval: ', bin_beta, bin_pval)            
                    
                bin_pvals_sens = [bin_pval_sens1, bin_pval_sens2]
                coef_dcts['bin'][intvn_map[intvn]][outcome_map[outcome]] = bin_beta
                se_dcts['bin'][intvn_map[intvn]][outcome_map[outcome]] = bin_se
                annot_dcts['bin'][intvn_map[intvn]][outcome_map[outcome]] = eval_pval(bin_pval, pval_thresh_1, pval_thresh_2, bin_pvals_sens)
                pval_dcts['bin'][intvn_map[intvn]][outcome_map[outcome]] = bin_pval
                print('bin_pvals_sens: ', bin_pvals_sens)


In [ ]:
plt.rcParams['font.family'] = 'Helvetica'
min_val = np.inf
max_val = -np.inf
for model in ['assoc', 'contemp', 'lagged', 'bin']:
    heat_dfs[model] = pd.DataFrame(coef_dcts[model])
    annot_dfs[model] = pd.DataFrame(annot_dcts[model])
    
    if heat_dfs[model].min().min() < min_val:
        min_val = heat_dfs[model].min().min()
    if heat_dfs[model].max().max() > max_val:
        max_val = heat_dfs[model].max().max()
    

model_maps = {'assoc': 'Association', 'contemp': 'Contemporaneous', 'lagged': 'Lagged',
             'bin': 'Binary ({d} Months)'.format(d=duration_months)}
    
for model in ['assoc', 'contemp', 'lagged', 'bin']:    
    plt.figure(figsize=(8, 6))  # Optional: Adjusts the size of the figure
    cmap = sns.diverging_palette(h_neg=130, h_pos=10, s=99, l=55, sep=3, as_cmap=True)

    ax = sns.heatmap(heat_dfs[model], annot=annot_dfs[model], center=0, fmt='s',
                     cmap = cmap,
                    annot_kws={"size": 25}, vmin=min_val, vmax=max_val, yticklabels=True, cbar=True)  # 'annot' annotates the boxes with the data values

    ax.set_yticklabels(ax.get_yticklabels(), rotation=0)

    # Display the heatmap
    plt.title(model_maps[model])
    plt.show()

In [ ]:
assoc_se_df = pd.DataFrame(se_dcts['assoc'])
assoc_coef_df = pd.DataFrame(coef_dcts['assoc'])
assoc_pval_df = pd.DataFrame(pval_dcts['assoc'])

lagged_se_df = pd.DataFrame(se_dcts['lagged'])
lagged_coef_df = pd.DataFrame(coef_dcts['lagged'])
lagged_pval_df = pd.DataFrame(pval_dcts['lagged'])

contemp_se_df = pd.DataFrame(se_dcts['contemp'])
contemp_coef_df = pd.DataFrame(coef_dcts['contemp'])
contemp_pval_df = pd.DataFrame(pval_dcts['contemp'])

bin_se_df = pd.DataFrame(se_dcts['bin'])
bin_coef_df = pd.DataFrame(coef_dcts['bin'])
bin_pval_df = pd.DataFrame(pval_dcts['bin'])

for col in lagged_se_df.columns:
    assoc_se_df = assoc_se_df.rename(columns={col:col+'_se'})
    assoc_coef_df = assoc_coef_df.rename(columns={col:col+'_pe'})
    assoc_pval_df = assoc_pval_df.rename(columns={col:col+'_pval'})
    
    lagged_se_df = lagged_se_df.rename(columns={col:col+'_se'})
    lagged_coef_df = lagged_coef_df.rename(columns={col:col+'_pe'})
    lagged_pval_df = lagged_pval_df.rename(columns={col:col+'_pval'})

    contemp_se_df = contemp_se_df.rename(columns={col:col+'_se'})
    contemp_coef_df = contemp_coef_df.rename(columns={col:col+'_pe'})
    contemp_pval_df = contemp_pval_df.rename(columns={col:col+'_pval'})

    bin_se_df = bin_se_df.rename(columns={col:col+'_se'})
    bin_coef_df = bin_coef_df.rename(columns={col:col+'_pe'})
    bin_pval_df = bin_pval_df.rename(columns={col:col+'_pval'})
    
assoc_df = pd.concat([assoc_se_df, assoc_coef_df, assoc_pval_df], axis=1)
lagged_df = pd.concat([lagged_se_df, lagged_coef_df, lagged_pval_df], axis=1)
contemp_df = pd.concat([contemp_se_df, contemp_coef_df, contemp_pval_df], axis=1)
bin_df = pd.concat([bin_se_df, bin_coef_df, bin_pval_df], axis=1)

assoc_df.index.name = 'Outcome'
lagged_df.index.name = 'Outcome'
contemp_df.index.name = 'Outcome'
bin_df.index.name = 'Outcome'

lagged_df

In [ ]:
save=True 
if save:
    prefix = '../results/' + run_name + '_' + units + '_' + str(datetime.now()).replace(' ', '_').replace(':', '_') + '/'
    os.makedirs(prefix)
    assoc_df.to_csv(prefix + 'assoc_df_{u}_all_primary.csv'.format(u=units))    
    lagged_df.to_csv(prefix + 'lagged_df_{u}_all_primary.csv'.format(u=units))
    contemp_df.to_csv(prefix + 'contemp_df_{u}_all_primary.csv'.format(u=units))
    bin_df.to_csv(prefix + 'bin_df_{u}_all_primary.csv'.format(u=units))
    
    annot_dfs['assoc'].to_csv(prefix + 'annot_assoc_df_{u}_all_primary.csv'.format(u=units))
    annot_dfs['contemp'].to_csv(prefix + 'annot_contemp_df_{u}_all_primary.csv'.format(u=units))
    annot_dfs['lagged'].to_csv(prefix + 'annot_lagged_df_{u}_all_primary.csv'.format(u=units))
    annot_dfs['bin'].to_csv(prefix + 'annot_bin_df_{u}_all_primary.csv'.format(u=units))

In [ ]:
prefix